In [17]:
import numpy as np
import csv
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import datasets, linear_model
from sklearn.cross_validation import KFold
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn import linear_model
#import pickle
#import io

## Import Data

In [27]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
profiles_data = pd.read_csv('profiles.csv')

train_set = pd.read_csv("train.csv", index_col=[0,1])
train_set_unstacked = train_set.unstack(level=-1)
train_set_unstacked.fillna(0, inplace=True)

In [5]:
train_KM = train
train_R = train

train = train_set_unstacked['plays']
train.loc[:,'user'] = train.index
train.index.name = None

#### Data Exploration

In [4]:
print "number of users missing age: %s" % sum(profiles_data['age'].isnull())
print "number of users missing sex: %s" % sum(profiles_data['sex'].isnull())
print "number of users missing country: %s" % sum(profiles_data['country'].isnull())
print "number of users with negative age: %s" % sum(profiles_data['age'] < 0 )
print "number of users with high age: %s" % sum(profiles_data['age'] > 120 )

number of users missing age: 44842
number of users missing sex: 19535
number of users missing country: 0
number of users with negative age: 40
number of users with high age: 4


#### Clean Data

In [6]:
df = pd.concat([profiles_data, pd.get_dummies(profiles_data['sex'], dummy_na=True)], axis=1)
df.rename(columns = lambda x: str(x), inplace=True)
df.rename(columns = {'nan': 'sex-nan'}, inplace=True)
del df['sex']

df['age-nan'] = np.isnan(df['age'])*1
nans = np.isnan(df['age'])
df.loc[nans,'age'] = 0
df['age-negative'] = (df['age']<0)*1
negatives = df['age-negative']==1
df.loc[negatives,'age'] = 0
df['age-toohigh'] = (df['age']>120)*1
toohighs = df['age-toohigh']==1
df.loc[toohighs,'age'] = 0

df = pd.concat([df, pd.get_dummies(df['country'], dummy_na=False)], axis=1)
del df['country']

df.index.name = None

In [7]:
data = pd.merge(train,df,on='user')
data = data.set_index('user')
data.index.name = None
data.head()

,000d90ec-d64c-48a1-b775-e726fd240e9f,000fc734-b7e1-4a01-92d1-f544261b43f5,0019749d-ee29-4a5f-ab17-6bfa11deb969,0039c7ae-e1a7-4a7d-9b49-0cbc716821a6,004e5eed-e267-46ea-b504-54526f1f377d,00565b31-14a3-4913-bd22-385eb40dd13c,00a9f935-ba93-4fc8-a33a-993abe9c936b,00eeed6b-5897-4359-8347-b8cd28375331,0103c1cc-4a09-4a5d-a344-56ad99a77193,0110e63e-0a9b-4818-af8e-41e180c20b9a,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, U.s.",Wallis and Futuna,Western Sahara,Yemen,Zambia,Zimbabwe
00000c289a1829a808ac09c00daf10bc3c4e223b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00001411dc427966b17297bf4d69e7e193135d89,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00007a47085b9aab8af55f52ec8846ac479ac4fe,0,0,0,0,0,0,0,0,0,456,...,0,0,0,0,0,0,0,0,0,0
0000c176103e538d5c9828e695fed4f7ae42dd01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Baseline
Overall Mean/Median + User Bias + Artist Popularity

#### K-Fold Cross Validation
Narrow down parameters (1. How to deal with negative predictions, 2. Mean v. Median)

In [64]:
kf = KFold(n=len(train), n_folds=5, shuffle=False, random_state=3)

kfold_predictions = []
MAE = []

for train_index, test_index in kf:
    
    X_train, X_test = train.ix[train_index, :], train.ix[test_index, :]
    
    overall_mean = X_train['plays'].mean()
    overall_median = X_train['plays'].median()

    subdf = X_train[['artist', 'plays']]
    #artist_means = subdf.groupby('artist').mean()
    #artist_means['avgplays'] = artist_means['plays'] - overall_mean
    #del artist_means['plays']
    #artist_means = pd.DataFrame.to_dict(artist_means)['avgplays']
    artist_medians = subdf.groupby('artist').median()
    artist_medians['medplays'] = artist_medians['plays'] - overall_median
    del artist_medians['plays']
    artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

    subdf = X_train[['user', 'plays']]
    #user_means = subdf.groupby('user').mean()
    #user_means['avgplays'] = user_means['plays'] - overall_mean
    #del user_means['plays']
    #user_means = pd.DataFrame.to_dict(user_means)['avgplays']
    user_medians = subdf.groupby('user').median()
    user_medians['medplays'] = user_medians['plays'] - overall_median
    del user_medians['plays']
    user_medians = pd.DataFrame.to_dict(user_medians)['medplays']
    
    subdf = X_train[['user', 'plays']]
    #user_means = subdf.groupby('user').mean()
    #user_means['avgplays'] = user_means['plays'] - overall_mean
    #del user_means['plays']
    #user_means = pd.DataFrame.to_dict(user_means)['avgplays']
    user_mins = subdf.groupby('user').min()
    user_mins['medplays'] = user_mins['plays']
    del user_mins['plays']
    user_mins = pd.DataFrame.to_dict(user_mins)['medplays']
    
    predictions = []
    for i, j in zip(X_test['user'], X_test['artist']):
        '''
        try:
            p = overall_mean + user_means[i] + artist_means[j]
            #if p < 0:
            #    p = 0
            predictions.append(p)
        except:
            p = overall_mean + 0 + artist_means[j]
            #if p < 0:
            #    p = 0
            predictions.append(p)
        '''
        try:
            p = overall_median + user_medians[i] + artist_medians[j]
            if p < 0:
                p = user_mins[i]
            predictions.append(p)
        except:
            p = overall_median + 0 + artist_medians[j]
            if p < 0:
                p = user_mins[i]
            predictions.append(p)
    #shift = min(predictions)
    #predictions = predictions + shift
    
    MAE.append(sum(abs(X_test['plays'] - predictions))/len(X_test))
    kfold_predictions.append(predictions)

In [65]:
print MAE # MEDIAN: for shifting negative predictions to min; mean = 
np.mean(MAE)

[142.37319802012368, 141.16751147165752, 141.41875491148201, 140.70536042004377, 140.74433065370175]


141.28183109540174

In [63]:
print MAE # MEDIAN: for shifting negative predictions to min /2; mean = 141.63912893689698
np.mean(MAE)

[142.73132674096618, 141.52288194030766, 141.77577226849394, 141.06299934653973, 141.10266438817752]


141.63912893689698

In [212]:
MAE # MEAN: for shifting every prediciton up to min; mean = 257.10431187965344

[257.28789827309532,
 258.15533086042217,
 256.69504440356701,
 255.94572232779623,
 257.43756353338642]

In [210]:
MAE # MEAN: for setting = 0; mean = 175.40618382357923

[176.25186801713923,
 174.94202683803979,
 175.06217330929474,
 175.41404116109535,
 175.36080979232705]

In [260]:
MAE # MEDIAN: for shifting every prediciton up to min; mean = 184.74907204172987

[186.90224450967014,
 184.03921543851035,
 184.39845913346113,
 184.14110168828623,
 184.26433943872149]

In [266]:
MAE # MEDIAN: for setting = 0; mean = 142.00740828329987

[143.100447554097,
 141.88868611162258,
 142.14390928094099,
 141.43150280217725,
 141.47249566766149]

#### Make Predictions using baseline model

In [216]:
# Write out test solutions - MEAN
overall_mean = train['plays'].mean()

subdf = train[['artist', 'plays']]
artist_means = subdf.groupby('artist').mean()
artist_means['avgplays'] = artist_means['plays'] - overall_mean
del artist_means['plays']
artist_means = pd.DataFrame.to_dict(artist_means)['avgplays']

subdf = train[['user', 'plays']]
user_means = subdf.groupby('user').mean()
user_means['avgplays'] = user_means['plays'] - overall_mean
del user_means['plays']
user_means = pd.DataFrame.to_dict(user_means)['avgplays']

predictions = []
for i, j in zip(test['user'], test['artist']):
    p = overall_mean + user_means[i] + artist_means[j]
    if p < 0:
        p = 0
    predictions.append(p)

# Save to a file named mean_adjust.csv
soln_file  = 'mean_adjust.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 174.23682

In [67]:
# Write out test solutions - MEDIAN
overall_median = train_R['plays'].median()

subdf = train_R[['artist', 'plays']]
artist_medians = subdf.groupby('artist').median()
artist_medians['medplays'] = artist_medians['plays'] - overall_median
del artist_medians['plays']
artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

subdf = train_R[['user', 'plays']]
user_medians = subdf.groupby('user').median()
user_medians['medplays'] = user_medians['plays'] - overall_median
del user_medians['plays']
user_medians = pd.DataFrame.to_dict(user_medians)['medplays']

subdf = train_R[['user', 'plays']]
user_mins = subdf.groupby('user').min()
user_mins['medplays'] = user_mins['plays']
del user_mins['plays']
user_mins = pd.DataFrame.to_dict(user_mins)['medplays']

predictions = []
for i, j in zip(test['user'], test['artist']):
    p = overall_median + user_medians[i] + artist_medians[j]
    if p < 0:
        p = user_mins[i]
    predictions.append(p)
    
    
# Save to a file named mean_adjust.csv
soln_file  = 'median_minshift.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 141.24741
# KAGGLE SCORE (shifting to min): 140.53659

### K-Means

Cluster the users into K classes

In [18]:
# Reading:  https://www.codementor.io/python/tutorial/build-data-products-django-machine-learning-clustering-user-preferences

# Truncate Data (dimension-reduction)
trunc = TruncatedSVD(n_components=100, random_state=4)
data_reduced = trunc.fit_transform(data)
data_reduced = pd.DataFrame(data_reduced)

data_KM = pd.merge(train,df,on='user')

In [155]:
# Cluster users into classes
K = 2 # Number of classes
kmeans = KMeans(n_clusters=K, init='random', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=3, copy_x=True, n_jobs=1)
kmeans.fit(data_reduced)

# Make dict of form User: Class
keys = list(data.index)
values = list(kmeans.labels_)
labels_dict = dict(zip(keys, values))

# Summary of results
# K = 100; 68 > 1; k-means++; avg k-fold 209.08475033596352
# K = 70; 49 > 1; k-means++; avg k-fold 207.22458452290766
# K = 50; 32 > 1; k-means++; avg k-fold 206.91192287314456
# K = 30; 15 > 1; k-means++; avg k-fold 203.56766892943034
# K = 10; 5 > 1; k-means++; avg k-fold 202.09096861815152
# K = 100; 90 > 1; random; avg k-fold 168.27485843741027
# K = 70; 64 > 1; random; avg k-fold 155.22775993231548
# K = 30; 26 > 1; random; avg k-fold 153.22289820225075
# K = 10; 8 > 1; random; avg k-fold 149.80848768499814
# K = 5; 4 > 1; random; avg k-fold 148.5971242801875
# K = 3; 2 > 1; random; avg k-fold 147.22173171754909

In [152]:
# enter labels into train and test dataframes

train_labels = []
for i in train_KM['user']:
    train_labels.append(labels_dict[i])
train_KM['label'] = train_labels


test_labels = [] 
for i in test['user']:
    test_labels.append(labels_dict[i])   
test['label'] = test_labels 

In [153]:
# Perform 5-fold cross validation

kf = KFold(n=len(train), n_folds=5, shuffle=False, random_state=3)

kfold_predictions = []
MAE = []

for train_index, test_index in kf:
    
    X_train, X_test = train_KM.ix[train_index, :], train_KM.ix[test_index, :]
    
    # Group Medians
    subdf = X_train[['plays', 'label']]
    group_medians = subdf.groupby('label').median()
    group_medians = pd.DataFrame.to_dict(group_medians)['plays']
    
    # User Medians
    subdf = X_train[['user', 'plays']]
    user_medians = subdf.groupby('user').median()
    user_medians = pd.DataFrame.to_dict(user_medians)['plays']
    
    #Artist Medians
    subdf = X_train[['artist', 'plays']]
    artist_medians = subdf.groupby('artist').median()
    artist_medians = pd.DataFrame.to_dict(artist_medians)['plays']
    
    subdf = train_R[['user', 'plays']]
    user_mins = subdf.groupby('user').min()
    user_mins['medplays'] = user_mins['plays']
    del user_mins['plays']
    user_mins = pd.DataFrame.to_dict(user_mins)['medplays']
    
    predictions = []
    for i, j, k in zip(X_test['label'], X_test['user'], X_test['artist']):
        try:
            p = group_medians[i] + (user_medians[j] - group_medians[i]) + (artist_medians[k] - group_medians[i])
            if p < 0:
                p = user_mins[i]
            predictions.append(p)
        except:
            try:
                p = group_medians[i] + 0 + (artist_medians[k] - group_medians[i])
                if p < 0:
                    p = user_mins[i]
                predictions.append(p)
            except:
                try:
                    p = 0 + user_medians[j] + artist_medians[k]
                    if p < 0:
                        p = user_mins[i]
                    predictions.append(p)
                except:
                    try:
                        p = group_medians[i] + 0 + (artist_medians[k] - group_medians[i])
                        if p < 0:
                            p = user_mins[i]
                        predictions.append(p)
                    except:
                        p = 0 + 0 + artist_medians[k]
                        if p < 0:
                            p = user_mins[i]
                        predictions.append(p)
                    
    MAE.append(sum(abs(X_test['plays'] - predictions))/len(X_test))
    kfold_predictions.append(predictions)

print np.mean(MAE)
print MAE

In [132]:
# Write out test solutions - K-means; initialization = ; K clusters = ;

# Group Medians
subdf = train_KM[['plays', 'label']]
group_medians = subdf.groupby('label').median()
group_medians = pd.DataFrame.to_dict(group_medians)['plays']

# User Medians
subdf = train_KM[['user', 'plays']]
user_medians = subdf.groupby('user').median()
user_medians = pd.DataFrame.to_dict(user_medians)['plays']

#Artist Medians
subdf = train_KM[['artist', 'plays']]
artist_medians = subdf.groupby('artist').median()
artist_medians = pd.DataFrame.to_dict(artist_medians)['plays']

# Prediction:
# Group Median + User Median + Artist Median
predictions = []
for i, j, k in zip(test['label'], test['user'], test['artist']):
    p = group_medians[i] + (user_medians[j] - group_medians[i]) + (artist_medians[k] - group_medians[i])
    if p < 0:
        p = 0
    predictions.append(p)

In [40]:
# Save to a file named mean_adjust.csv
soln_file  = 'kmeans_100_random.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 165.70278 (K=100, random)

### Regression

In [157]:
# MEDIAN / MEAN
# Apply min when predictions are < 0

kf = KFold(n=len(train), n_folds=5, shuffle=False, random_state=3)
kfold_predictions = []
MAE = []

for train_index, test_index in kf:
    
    # Dataset for Regression
    train_R = train_KM

    X_train, X_test = train_R.ix[train_index, :], train_R.ix[test_index, :]
    
    subdf = X_train[['artist', 'plays']]
    artist_medians = subdf.groupby('artist').median()
    artist_medians['medplays'] = artist_medians['plays']
    del artist_medians['plays']
    artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

    subdf = X_train[['user', 'plays']]
    user_medians = subdf.groupby('user').median()
    user_medians['medplays'] = user_medians['plays']
    del user_medians['plays']
    user_medians = pd.DataFrame.to_dict(user_medians)['medplays']
    
    subdf = X_train[['user', 'plays']]
    user_mins = subdf.groupby('user').min()
    user_mins['medplays'] = user_mins['plays']
    del user_mins['plays']
    user_mins = pd.DataFrame.to_dict(user_mins)['medplays']

    # Training Dataset
    user_m = []
    artist_m = []
    user_min = []
    for i, j in zip(X_train['user'], X_train['artist']):
        try:
            user_m.append(user_medians[i])
        except:
            user_m.append(0)
        try:
            user_min.append(user_mins[i])
        except:
            user_min.append(0)
        artist_m.append(artist_medians[j])      

    X_train['user_median'] = user_m
    X_train['artist_median'] = artist_m
    X_train['user_min'] = user_min
    
    X_train1 = X_train


    # Test Dataset
    user_m = []
    artist_m = []
    user_min = []
    for i, j in zip(X_test['user'], X_test['artist']):
        try:
            user_m.append(user_medians[i])
        except:
            user_m.append(0)
        try:
            user_min.append(user_mins[i])
        except:
            user_min.append(0)
        artist_m.append(artist_medians[j])

    X_test['user_median'] = user_m
    X_test['artist_median'] = artist_m
    X_test['user_min'] = user_min
    
    X_test1 = X_test

    # Make X_train, X_test, Y_train
    Y_train = X_train['plays']
    X_train = X_train[['user_median', 'artist_median']]
    Y_test = X_test['plays']
    X_test = X_test[['user_median', 'artist_median']]

    # Predictions
    regr = linear_model.LinearRegression()
    regr.fit(X_train, Y_train)
    predictions = regr.predict(X_test)
    
    for p in range(len(predictions)):
        if predictions[p] < 0:
            predictions[p] = np.array(X_test1['user_min'])[p]
    
    print regr.coef_

    MAE.append(sum(abs(Y_test - predictions))/len(Y_test))
    kfold_predictions.append(predictions) 

[ 1.34923353  1.06158177]
[ 1.35553713  1.06510331]
[ 1.35267186  1.05914421]
[ 1.34861051  1.07320877]
[ 1.35259899  1.0678157 ]


In [158]:
print MAE
print np.mean(MAE)

[161.94307891326221, 161.04739308964915, 161.12305526194345, 160.71741622877846, 160.77141202351112]
161.120471103


In [159]:
# REGRESSION

subdf = train_R[['artist', 'plays']]
artist_medians = subdf.groupby('artist').median()
artist_medians['medplays'] = artist_medians['plays']
del artist_medians['plays']
artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

subdf = train_R[['user', 'plays']]
user_medians = subdf.groupby('user').median()
user_medians['medplays'] = user_medians['plays']
del user_medians['plays']
user_medians = pd.DataFrame.to_dict(user_medians)['medplays']

subdf = train_R[['user', 'plays']]
user_mins = subdf.groupby('user').min()
user_mins['medplays'] = user_mins['plays']
del user_mins['plays']
user_mins = pd.DataFrame.to_dict(user_mins)['medplays']

In [162]:
# Training Dataset
user_m = []
artist_m = []
user_min = []
for i, j in zip(train_R['user'], train_R['artist']):
    user_m.append(user_medians[i])
    artist_m.append(artist_medians[j])
    user_min.append(artist_medians[j])
    
train_R['user_median'] = user_m
train_R['artist_median'] = artist_m
train_R['user_min'] = user_min

train_R1 = train_R


# Test Dataset
user_m = []
artist_m = []
user_min = []
for i, j in zip(test['user'], test['artist']):
    user_m.append(user_medians[i])
    artist_m.append(artist_medians[j])
    user_min.append(artist_medians[j])
    
test_R = test
test_R['user_median'] = user_m
test_R['artist_median'] = artist_m
test_R['user_min'] = user_min

test_R1 = test_R

# Make X_train, X_test, Y_train
X_train = train_R[['user_median', 'artist_median']]
Y_train = train_R['plays']
X_test = test_R[['user_median', 'artist_median']]

In [163]:
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)
predictions = regr.predict(X_test)

print regr.coef_

for p in range(len(predictions)):
    if predictions[p] < 0:
        predictions[p] = np.array(test_R1['user_min'])[p]

[ 1.36811555  1.07681351]


In [164]:
predictions

array([ 173.55398645,  200.69731239,  317.18502178, ...,  263.52717276,
        367.44367985,  133.05394673])

In [165]:
# Write out test solutions - MEDIAN
soln_file  = 'regression1.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 161.34539

### Ridge Regression

In [174]:
# MEDIAN

kf = KFold(n=len(train), n_folds=5, shuffle=False, random_state=3)
kfold_predictions = []
MAE = []

for train_index, test_index in kf:
    
    # Dataset for Regression
    train_R = train_KM

    X_train, X_test = train_R.ix[train_index, :], train_R.ix[test_index, :]
    
    subdf = X_train[['artist', 'plays']]
    artist_medians = subdf.groupby('artist').median()
    artist_medians['medplays'] = artist_medians['plays']
    del artist_medians['plays']
    artist_medians = pd.DataFrame.to_dict(artist_medians)['medplays']

    subdf = X_train[['user', 'plays']]
    user_medians = subdf.groupby('user').median()
    user_medians['medplays'] = user_medians['plays']
    del user_medians['plays']
    user_medians = pd.DataFrame.to_dict(user_medians)['medplays']
    
    subdf = X_train[['user', 'plays']]
    user_mins = subdf.groupby('user').min()
    user_mins['medplays'] = user_mins['plays']
    del user_mins['plays']
    user_mins = pd.DataFrame.to_dict(user_mins)['medplays']
    
    # Training Dataset
    user_m = []
    artist_m = []
    user_min = []
    for i, j in zip(X_train['user'], X_train['artist']):
        try:
            user_m.append(user_medians[i])
        except:
            user_m.append(0)
        try:
            user_min.append(user_mins[i])
        except:
            user_min.append(0)
        artist_m.append(artist_medians[j])      

    X_train['user_median'] = user_m
    X_train['artist_median'] = artist_m
    X_train['user_min'] = user_min
    
    X_train1 = X_train


    # Test Dataset
    user_m = []
    artist_m = []
    user_min = []
    for i, j in zip(X_test['user'], X_test['artist']):
        try:
            user_m.append(user_medians[i])
        except:
            user_m.append(0)
        try:
            user_min.append(user_mins[i])
        except:
            user_min.append(0)
        artist_m.append(artist_medians[j])

    X_test['user_median'] = user_m
    X_test['artist_median'] = artist_m
    X_test['user_min'] = user_min
    
    X_test1 = X_test

    # Make X_train, X_test, Y_train
    Y_train = X_train['plays']
    X_train = X_train[['user_median', 'artist_median']]
    Y_test = X_test['plays']
    X_test = X_test[['user_median', 'artist_median']]

    # Predictions
    ridge_regr = linear_model.Ridge(alpha=10, fit_intercept=True)
    ridge_regr.fit(X_train, Y_train)
    predictions = ridge_regr.predict(X_test)
    
    for p in range(len(predictions)):
        if predictions[p] < 0:
            predictions[p] = np.array(test_R1['user_min'])[p]

    MAE.append(sum(abs(Y_test - predictions))/len(Y_test))
    kfold_predictions.append(predictions) 

In [175]:
print MAE # alpha = 10 Median;
print np.mean(MAE)

[165.23366052719109, 164.4411606532432, 164.45699510820177, 164.02331745432804, 164.14289619652567]
164.459605988


In [173]:
print MAE # alpha = 1 Median;
print np.mean(MAE)

[165.23366053966097, 164.44116066578425, 164.45699512055506, 164.02331746702006, 164.1428962090848]
164.459606


In [176]:
# Training Dataset
user_m = []
artist_m = []
user_min = []
for i, j in zip(train_R['user'], train_R['artist']):
    user_m.append(user_medians[i])
    artist_m.append(artist_medians[j])
    user_min.append(artist_medians[j])
    
train_R['user_median'] = user_m
train_R['artist_median'] = artist_m
train_R['user_min'] = user_min

train_R1 = train_R


# Test Dataset
user_m = []
artist_m = []
user_min = []
for i, j in zip(test['user'], test['artist']):
    user_m.append(user_medians[i])
    artist_m.append(artist_medians[j])
    user_min.append(artist_medians[j])
    
test_R = test
test_R['user_median'] = user_m
test_R['artist_median'] = artist_m
test_R['user_min'] = user_min

test_R1 = test_R

# Make X_train, X_test, Y_train
X_train = train_R[['user_median', 'artist_median']]
Y_train = train_R['plays']
X_test = test_R[['user_median', 'artist_median']]

In [177]:
regr = linear_model.Ridge(alpha = 1)
regr.fit(X_train, Y_train)
predictions = regr.predict(X_test)

print regr.coef_

for p in range(len(predictions)):
    if predictions[p] < 0:
        predictions[p] = np.array(test_R1['user_min'])[p]

[ 1.32484819  1.09451686]


In [178]:
predictions

array([ 173.02151765,  187.56096094,  313.16440966, ...,  271.00314363,
        380.385284  ,  132.8707769 ])

In [179]:
# Write out test solutions - MEDIAN
soln_file  = 'ridge_regression.csv'
test_file  = 'test.csv'

with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            p = predictions[int(id)-1]

            soln_csv.writerow([id, p])
# KAGGLE SCORE: 